# Cross Strata Detectron2 Fast RCNN

## Experiments Notebook

Authors: Gabrielle Melli, Andrew Foerder

Date: 9/23/2024

# Purpose

The purpose of this notebook is to run training experiements using a Detectron2 COCO detection fast rcnn model to recognize Cross-Strata features in ariel images of Earth and Mars.

# Methods

## User Settings

In [1]:
# Connect to GDrive to access image files and labels
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/__Shared/Andrew/working_directory'
%pwd

/content/gdrive/.shortcut-targets-by-id/1Bhg_-iEsxa3Aq4v-SLxPVOpKrY5TpKaq/Andrew/working_directory


'/content/gdrive/.shortcut-targets-by-id/1Bhg_-iEsxa3Aq4v-SLxPVOpKrY5TpKaq/Andrew/working_directory'

In [3]:
import pprint
from config_utils import Config

In [4]:
# TODO automate this
label_date_recent = '9-16-24.json'

In [5]:
c = Config()
c.get_paths()
pprint.pp(c.paths)

{'white_sands_heli': {'dir_to_split': '/content/gdrive/MyDrive/__Shared/Andrew/working_directory/data_sources/white_sands_heli/xstrata',
                      'raw_anno_path': '/content/gdrive/MyDrive/__Shared/Andrew/working_directory/data_sources/white_sands_heli/xstrata/labels',
                      'output_dir': '/content/output/data/white_sands_heli',
                      'output_dir_train': '/content/output/data/white_sands_heli/train',
                      'output_dir_train_img': '/content/output/data/white_sands_heli/train/images',
                      'output_dir_train_anno': '/content/output/data/white_sands_heli/train/json_annotation_train.json',
                      'output_dir_val': '/content/output/data/white_sands_heli/val',
                      'output_dir_val_img': '/content/output/data/white_sands_heli/val/images',
                      'output_dir_val_anno': '/content/output/data/white_sands_heli/val/json_annotation_train.json'},
 'white_sands_sat': {'dir_to_spl

## Setup

In [6]:
%%capture
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

!python -m pip install split-folders
!pip install pycocotools

In [7]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.4 ; cuda:  cu121
detectron2: 0.6


In [8]:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import pandas as pd
import numpy as np
import json, cv2, random

from google.colab.patches import cv2_imshow

# import libraries that allow for dataset generation
import splitfolders

from pycocotools.cocoeval import COCOeval

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, DatasetEvaluators

# import functions written for this project
from data_processing.data_load import get_anno_path, split_json_train_val

In [9]:
# Split the image folders into 'train' and 'val' folders
for dataset in c.cfg['datasets']:
    splitfolders.ratio(c.paths[dataset]['dir_to_split'],
                       output=c.paths[dataset]['output_dir'],
                       seed=1420,
                       ratio=(0.7, 0.3))

Copying files: 237 files [00:08, 27.24 files/s] 
Copying files: 25 files [00:34,  1.36s/ files]


## Dataset Loading

In [10]:
# get the original labels, split them, and register datasets
t = []
v = []

for dataset in c.cfg['datasets']:
    anno_path = get_anno_path(c.paths[dataset]['raw_anno_path'], label_date_recent)

    split_json_train_val(json_filepath = anno_path,
                         train_image_dir = c.paths[dataset]['output_dir_train_img'],
                         val_image_dir = c.paths[dataset]['output_dir_val_img'],
                         train_json_filepath = c.paths[dataset]['output_dir_train_anno'],
                         val_json_filepath = c.paths[dataset]['output_dir_val_anno'])

    train_dataset = dataset + '_train'
    t.append(train_dataset)
    val_dataset = dataset + '_val'
    v.append(val_dataset)

    register_coco_instances(train_dataset, {}, c.paths[dataset]['output_dir_train_anno'], c.paths[dataset]['output_dir_train_img'])
    register_coco_instances(val_dataset, {}, c.paths[dataset]['output_dir_val_anno'], c.paths[dataset]['output_dir_val_img'])
    xstrata_metadata = MetadataCatalog.get(train_dataset)
    
train_datasets = tuple(t)
val_datasets = tuple(v)


## Model Training

In [ ]:
# initiate a detectron2 config object
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(c.cfg['model_settings']['model_zoo_cfg']))
cfg.DATASETS.TRAIN = train_datasets
cfg.DATASETS.TEST = (val_datasets[0])

# load settings from config
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(c.cfg['model_settings']['model_zoo_cfg'])
cfg.DATALOADER.NUM_WORKERS = c.cfg['model_settings']['dataloader__num_workers']

cfg.SOLVER.IMS_PER_BATCH = c.cfg['model_settings']['solver__ims_per_batch']
cfg.SOLVER.BASE_LR = c.cfg['model_settings']['solver__base_learning_rate']
cfg.SOLVER.MAX_ITER = c.cfg['model_settings']['solver__max_iter']                                               # comment from detectron2 tutorial: 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = c.cfg['model_settings']['solver__steps']                                                     # comment from detectron2 tutorial: do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = c.cfg['model_settings']['model__roi_heads__batch_size_per_image']    # comment from detectron2 tutorial: The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = c.cfg['model_settings']['model__roi_heads__num_classes']
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = c.cfg['model_settings']['model__roi_heads__score_threshold_test']

# creates directories to output results
cfg.OUTPUT_DIR = c.cfg['paths']['google_colab']['output_path']
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
# define detectron2 trainer object using the cfg object we defined above
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(val_datasets[1], cfg, False, output_dir=cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, val_datasets[1])
inference_on_dataset(trainer.model, val_loader, evaluator)